# Lab 2

Getting the best models on the planet to generate an SVG

Inspired by the legendary Simon Willison's Pelican riding a bike test

In [ ]:
from dotenv import load_dotenv
from IPython.display import Markdown, display
from datetime import datetime
from litellm import acompletion
import asyncio
import time
from revealer import reveal

load_dotenv(override=True)

In [ ]:
challenge = "A panda waterskiing"
prompt = f"Generate an SVG of {challenge}. Respond with the SVG only, no code blocks."
messages = [{"role": "user", "content": prompt}]


async def artist(model, effort=None):
    try:
        start = datetime.now()
        response = await acompletion(model=model, messages=messages, reasoning_effort=effort)
        result = response.choices[0].message.content
        end = datetime.now()
        elapsed = (end - start).total_seconds()
        heading = f"### {model}\n**Time:** {elapsed // 60:.0f} min {elapsed % 60:.0f} s\n\n"
    except Exception as e:
        print(f"Model {model} failed: {e}")
        heading = f"### {model}\n**Error:** {e}\n\n"
        return heading, None
    return heading, result

In [ ]:
artists = [
    artist("groq/openai/gpt-oss-120b"),
    artist("openai/gpt-5-nano", effort="low"),
    artist("deepseek/deepseek-reasoner"),
    artist("openrouter/moonshotai/kimi-k2-thinking"),
    artist("xai/grok-4-1-fast-reasoning"),
    artist("anthropic/claude-opus-4-5"),
    artist("openai/gpt-5.1", effort="high"),
    artist("gemini/gemini-3-pro-preview")
]

In [ ]:
results = await asyncio.gather(*artists)

In [ ]:
for result in results:
    try:
        display(Markdown(result[0]))
        reveal(result[1])
        time.sleep(5)
    except Exception as e:
        print(f"Error displaying result: {e}")